In [1]:
%load_ext autoreload

In [8]:
import altair as alt
import polars as pl

%autoreload
from altair_utils import color_sf_bayarea
from utils import line_plot_normalized

In [3]:
df = (
    pl.read_csv(
        r"Q:\Model Research\downtown_today\census\ACS\B19081-MeanHouseholdIncomeOfQuintiles.csv"
    )
    .with_columns(
        pl.col("geography").replace(
            {"San Jose-San Francisco-Oakland CSA": "SJ-SF-Oakland CSA"}
        )
    )
    .rename({"mean household income of quintiles": "mean household income"})
    .sort("year")
)

In [ ]:
def plot_quintile_mean(df, quintile):
    return (
        alt.Chart(
            df.filter(pl.col("quintile") == quintile).with_columns(
                date=pl.date(pl.col("year"), 1, 1)
            )
        )
        .mark_line(strokeWidth=1.5)
        .encode(
            x=alt.X(
                "date",
                title="year",
            ),
            y=alt.Y(
                "mean household income", title="mean household income of quintiles"
            ),
            color=alt.Color("geography").scale(
                domain=["San Francisco", "SJ-SF-Oakland CSA"],
                range=color_sf_bayarea,
            ),
            # strokeDash="Quintile",
            tooltip=["year", "geography", "quintile", "mean household income"],
        )
    )


chart = alt.layer(*(plot_quintile_mean(df, i) for i in range(1, 6))).interactive()
chart.save("output/Links/income.png")
chart.properties(width="container").save("output/Links/income.html")
chart

In [ ]:
sf_normalized_df, sf_normalized_chart = line_plot_normalized(
    df.filter(
        # filter out the regional lines; too cluttered otherwise
        (pl.col("geography") == "San Francisco") & (pl.col("year") > 2018)
    ),
    x="year",
    y="mean household income",
    title="San Francisco mean household income of quintiles",
    color="quintile:O",
    # mark_line_args={"strokeWidth": 1.5},
    # chart_args={"strokeDash": "Quintile"},
    tooltip=["year", "geography", "quintile", "mean household income"],
)
sf_normalized_df.write_csv("output/data/income-sf-2019norm.csv")
sf_normalized_chart.save("output/Links/income-sf-2019norm.png")
sf_normalized_chart.properties(width="container").save(
    "output/Links/income-sf-2019norm.html"
)
sf_normalized_chart

In [ ]:
csa_normalized_df, csa_normalized_chart = line_plot_normalized(
    df.filter(
        # filter out the regional lines; too cluttered otherwise
        (pl.col("geography") == "SJ-SF-Oakland CSA") & (pl.col("year") > 2018)
    ),
    x="year",
    y="mean household income",
    title="SJ-SF-Oakland CSA mean household income of quintiles",
    color="quintile:O",
    # mark_line_args={"strokeWidth": 1.5},
    # chart_args={"strokeDash": "Quintile"},
    tooltip=["year", "geography", "quintile", "mean household income"],
)
csa_normalized_df.write_csv("output/data/income-csa-2019norm.csv")
csa_normalized_chart.save("output/Links/income-csa-2019norm.png")
csa_normalized_chart.properties(width="container").save(
    "output/Links/income-csa-2019norm.html"
)
csa_normalized_chart